In [26]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import random
from collections import Counter
import simulai
from simulai.regression import DenseNetwork
from simulai.models import DeepONet
# from simulai.optimization import Optimizer

### Data

In [27]:
# data_path = os.environ['DATASET_PATH']
# datasets = np.load(data_path)
# data_path

with h5py.File("c:\\Users\\kaoid\\My Drive\\Estudo\\Poli\\Disciplinas\\PINNs\\acousticPINNs\\dataset.h5", 'r') as f:
  data = list(f["standardData"]) # read standardized data
  # data = list(f["unitData"]) # read unit normalized data

# dataset organized as list of 4 numpy.ndarray elements:
# x coordinates, y coordinates, time steps, pressure values.
# reorganize data in [nodeQuantity * time steps, 4] matrix with
# one information per column
dataMatrix = np.concatenate(([data[i][:, None] for i in range(4)]), axis = 1)
print(f"Dataset shape: {dataMatrix.shape}")
# count = dict(Counter(data[2]))
# for k in count:
#   print(k, count[k])

Dataset shape: (391036, 4)


In [28]:
# input_dataset_raw is [sensor, case]
# input_dataset_raw = datasets['input_dataset']

n_cases = 30
n_sensors = 1000
# choose n_sensors random points in physical domain (which has 3371 nodes)
randNodeIDs = random.sample(range(3372), k = n_sensors)
# [sensor, case] matrix for input_dataset. Each line contains
# the pressure history of a specific "sensor" for the first n_cases time steps.
input_dataset = np.concatenate(
    ([dataMatrix[randNodeIDs + np.tile(3371 * step, 1), 2][:, None] for step in range(n_cases)]),
    axis = 1
)
print(f"Input dataset shape: {input_dataset.shape}")

# output_dataset_raw is [time step, ?, case]
# output_dataset_raw = datasets['output_dataset']

timeStepAmount = int(dataMatrix.shape[0] / 3371) # number of time steps
# same as input_dataset, but for the LAST n_cases time steps
output_dataset = np.concatenate(
    ([dataMatrix[
        randNodeIDs + np.tile(3371 * step, 1), 2
    ][:, None] for step in range(timeStepAmount - n_cases, timeStepAmount)]),
    axis = 1
)
print(f"Output dataset shape: {output_dataset.shape}")
# time_raw = datasets['time']

Input dataset shape: (1000, 30)
Output dataset shape: (1000, 30)


### Parameters

In [29]:
# time_interval = [0, 120]
# n_cases_test = 20
# n_time_samples = 50
latent_dim = 500
# percentage of dataset used for testing
testPercentage = 0.3
# n_vars = 2 not used
activation = "relu"
trunk_layers_units = [50, 50, 50]
branch_layers_units = [50, 50, 50]
n_inputs = 1
lr = 1e-3
lambda_1 = 0.0
lambda_2 = 1e-5
n_epochs = 10

### Times and sensors

In [30]:
# set apart time values up to last time step
# time_ = time_raw[time_raw <= time_interval[-1]] # datasets['time']
# sorted 'n_time_samples' random indices of 'time_'
# time_indices = sorted(np.random.choice(time_.shape[0], n_time_samples))
# time = time_[time_indices]

# time step values
time = np.asarray(list(dict(Counter(dataMatrix[:, 2])).keys()))
print(f"timeStepAmount: {timeStepAmount}")
print(f"'time' vector shape: {time.shape}")
# sensors_indices = np.linspace(0, time_.shape[0], n_sensors).astype(int)

timeStepAmount: 116
'time' vector shape: (116,)


### Split datasets

In [31]:
# output_dataset_raw is [time step, ?, case]
# output_dataset_train = output_dataset_raw[:, :, :n_cases]
# output_dataset_test = output_dataset_raw[:, :, n_cases:]

# output_dataset is [sensor, case]
output_dataset_train = output_dataset[:, :round(n_cases * (1 - testPercentage))]
output_dataset_test = output_dataset[:, round(n_cases * (1 - testPercentage)):]
print(f"output_dataset_train shape: {output_dataset_train.shape}")
print(f"output_dataset_test shape: {output_dataset_test.shape}")

# input_dataset_raw is [sensor, case]
# input_dataset_train = input_dataset_raw[:, :n_cases]
# input_dataset_test = input_dataset_raw[:, n_cases:]

# input_dataset is [sensor, case]
input_dataset_train = input_dataset[:, :round(n_cases * (1 - testPercentage))]
input_dataset_test = input_dataset[:, round(n_cases * (1 - testPercentage)):]
print(f"input_dataset_train shape: {input_dataset_train.shape}")
print(f"input_dataset_test shape: {input_dataset_test.shape}")

output_dataset_train shape: (1000, 21)
output_dataset_test shape: (1000, 9)
input_dataset_train shape: (1000, 21)
input_dataset_test shape: (1000, 9)


### Instants and sensors

Set apart data from desired instants and sensors

In [32]:
# output_dataset_raw is [time step, ?, case]
# output_dataset_time_sampled = output_dataset_train[time_indices, ...]

output_dataset_time_sampled = output_dataset_train[:, None, :]
print(output_dataset_time_sampled.shape)

# input_dataset_raw is [sensor, case]
# input_dataset_sensor_sampled = input_dataset_train[sensors_indices, ...][:, None, :]

input_dataset_sensor_sampled = input_dataset_train[:, None, :]
print(input_dataset_sensor_sampled.shape)
verify_case_index = 100

(1000, 1, 21)
(1000, 1, 21)


### Inputs and outputs

In [33]:
# [time step, ?, case] -> transpose -> [case, time step, ?] -> reshape -> [case * time step, -1]
# output_target = output_dataset_time_sampled.transpose(2, 0, 1).reshape(n_cases * n_time_samples, -1)

trainSize = round(n_cases * (1 - testPercentage))
output_target = output_dataset_time_sampled.transpose(2, 0, 1).reshape(trainSize, -1)

# [sensor, None, case] -> transpose -> [case, None, sensor]
# repeat for (1, n_time_samples, 1)
# reshape to [n_cases * n_time_samples, -1]
# input_branch = np.tile(input_dataset_sensor_sampled.transpose(2, 1, 0), (1, n_time_samples, 1)).reshape(n_cases * n_time_samples, -1)
input_branch = np.tile(
  input_dataset_sensor_sampled.transpose(2, 1, 0),
  (1, timeStepAmount, 1)
).reshape(trainSize, -1)

# repeat time instants for (n_cases, 1)
input_trunk = np.tile(time[:, None], (n_cases, 1))

print(f"output_target shape: {output_target.shape}")
print(f"input_branch shape: {input_branch.shape}")
print(f"input_trunk shape: {input_trunk.shape}")

output_target shape: (21, 1000)
input_branch shape: (21, 116000)
input_trunk shape: (3480, 1)


### Setup NNs and optimizer

In [34]:
# Configuration for the fully-connected network
config_trunk = {
  'layers_units': trunk_layers_units,  # Hidden layers
  'activations': activation,
  'input_size': n_inputs,
  'output_size': latent_dim,
  'name': 'trunk_net'
}

 # Configuration for the fully-connected network
config_branch = {
  'layers_units': branch_layers_units,  # Hidden layers
  'activations': activation,
  'input_size': n_sensors,
  'output_size': latent_dim,
  'name': 'branch_net'
}

# Instantiating and training the surrogate model
trunk_net = DenseNetwork(**config_trunk)

# Instantiating and training the surrogate model
branch_net = DenseNetwork(**config_branch)

trunk_net.summary()
branch_net.summary()

optimizer_config = {'lr': lr}

# Maximum derivative magnitudes to be used as loss weights
maximum_values = (1/np.linalg.norm(output_target, 2, axis=0)).tolist()

params = {'lambda_1': lambda_1, 'lambda_2': lambda_2, 'weights': maximum_values}

input_data = {'input_branch': input_branch, 'input_trunk': input_trunk}

# The DeepONet receives the two instances in order to construct 
# the trunk and the branch components
op_net = DeepONet(trunk_network=trunk_net, branch_network=branch_net, var_dim=2, model_id="LotkaVolterra")

Summary of the network properties:
Linear operations layers:

[ Linear(in_features=1, out_features=50, bias=True),
  Linear(in_features=50, out_features=50, bias=True),
  Linear(in_features=50, out_features=50, bias=True),
  Linear(in_features=50, out_features=500, bias=True)]


Activations layers:

['relu', 'relu', 'relu', 'identity']


Initializations at each layer:

['xavier', 'xavier', 'xavier', 'xavier']
Summary of the network properties:
Linear operations layers:

[ Linear(in_features=1000, out_features=50, bias=True),
  Linear(in_features=50, out_features=50, bias=True),
  Linear(in_features=50, out_features=50, bias=True),
  Linear(in_features=50, out_features=500, bias=True)]


Activations layers:

['relu', 'relu', 'relu', 'identity']


Initializations at each layer:

['xavier', 'xavier', 'xavier', 'xavier']


In [ ]:
optimizer = Optimizer('adam', params=optimizer_config)
optimizer.fit(op_net, input_data=input_data, target_data=output_target,
                      n_epochs=n_epochs, loss="wrmse", params=params)

In [25]:
n_tests_choices = 100
test_indices = np.random.choice(n_cases_test, n_tests_choices)
time_test = np.linspace(0, time_interval[-1], 2000)[:,None]

for index in test_indices[::10]:
    
    target_test = output_dataset_test[:, :, index]
    input_test_ = input_dataset_test[None, sensors_indices, index]
    input_test = np.tile(input_test_, (2000, 1))
    evaluation = op_net.eval(trunk_data=time_test, branch_data=input_test)
    
    plt.plot(time_raw, target_test[:,0], label="Exact")
    plt.plot(time_test, evaluation[:,0], label="Approximated")
    plt.legend()
    plt.xlim(0, 120)
    plt.show()

NameError: name 'n_cases_test' is not defined